In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder
import numpy as np
from data_work import data_split
from data_work import processed
from data_work import analyse_dataset
from data_work import set_data
from sklearn.model_selection import train_test_split
import pandas as pd
data=pd.read_csv("C:\\NCKH\\FLMODEL\\datanonid.csv",index_col=None)


In [4]:
id=[16,17,21,22,23,24]
col=data.columns
for idx in id:
    data=data.drop(col[idx],axis=1)
data=data.drop(columns='attack')
data=data.drop(columns='category')
data = processed(data,"subcategory ")

In [5]:

traindata,test_data=train_test_split(data,test_size=0.4,random_state=42,stratify=data['subcategory '])
data = traindata.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
client_data,server_data=train_test_split(traindata,test_size=0.001,random_state=42)
client_data = client_data.reset_index(drop=True)
server_data = server_data.reset_index(drop=True)

In [6]:

testdata_list=set_data([test_data])[0]

#chia data thành list gồm 2 phàn tử [0] là X, [1] là y

len X 2811, len y 2811


# iid dataset setup 

In [7]:
from model import Net

##Create model function

In [8]:
import torch
import torch.nn as nn

In [9]:
trainloaders,testloaders=data_split(client_data,"subcategory ",8,0.1,IID=1)

prepairing IID dataset
len of client data :4210, len of each subdata : 526
client number 1 : train(474), val(52)
client number 2 : train(474), val(52)
client number 3 : train(474), val(52)
client number 4 : train(474), val(52)
client number 5 : train(474), val(52)
client number 6 : train(474), val(52)
client number 7 : train(474), val(52)
client number 8 : train(474), val(52)


In [10]:
names=["subcategory "]
a=analyse_dataset(trainloaders[7],names)
print(a)

thống kê nhãn subcategory :
3 62
1 67
2 62
6 62
7 68
4 75
5 69
0 9
[3, 1, 2, 6, 7, 4, 5, 0]


In [11]:
def semantic_backdoor(data,feature):
    threshold = data[feature].max()
    data.loc[data[feature] >= (threshold*0.75),"subcategory "]= 0
    return data

##Client

In [12]:
class client():
    def __init__(self,cid,net):
        self.cid=cid 
        self.model=net
        self.trainset=trainloaders[cid]
        self.client_label=analyse_dataset(trainloaders[cid],["subcategory "])
        self.discrete_columns=list(trainloaders[cid].columns)

    def fit(self ):
        # Read values from config
        self.model.fit(self.trainset, self.discrete_columns)
        return "ok"
    
    def Gen_fake_data(self,n_samples):
        synthetic_data = self.model.sample(n_samples)
        return synthetic_data



In [13]:
config = {
    "server_round": 3,
    "local_epochs": 5
    
}

In [14]:
from ctgan import CTGAN
from ctgan import load_demo

def client_fn(cid: str,config): 
    server_round = config["server_round"]
    local_epochs = config["local_epochs"]
    net = CTGAN(epochs=local_epochs)
    print(cid)
    return client(cid, net)

In [15]:
input_shape=data.shape[1]
data

,subcategory,pkSeqID,stime,flgs,proto,saddr,sport,daddr,dport,pkts,...,sum,min,max,spkts,dpkts,sbytes,dbytes,rate,srate,drate
0,1,4867,5223,3,2,1,3838,25,900,19,...,2433,713,2246,16,5,503,112,1131,1210,622
1,5,702,684,0,2,3,1908,20,493,1,...,60,60,60,0,1,0,1,2434,0,0
2,5,887,845,0,2,5,4787,18,533,1,...,23,0,23,1,0,30,0,0,0,0
3,4,2134,1940,0,2,3,3079,20,285,1,...,203,203,203,0,1,0,1,2265,0,0
4,5,430,423,0,2,3,1908,20,977,1,...,68,68,68,0,1,0,1,2424,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4210,5,413,407,0,2,3,1908,20,796,1,...,200,200,200,0,1,0,1,2268,0,0
4211,4,2750,2361,0,2,3,3079,20,272,1,...,346,368,340,0,1,0,1,2117,0,0
4212,2,3723,6296,0,2,5,4421,17,600,1,...,156,156,156,0,1,2,1,2320,0,0
4213,1,4317,4683,7,2,1,3699,25,900,8,...,2298,1460,2401,5,3,444,108,1321,1231,795


In [16]:
df_label_0 = data[data['subcategory '] == 0]
print(df_label_0)

      subcategory   pkSeqID  stime  flgs  proto  saddr  sport  daddr  dport  \
25               0     3918   5383     0      2      3   1912     20    372   
42               0     3984   5423     4      2      3   1961     20    372   
206              0     3951   5416     0      2      3   1952     20    372   
217              0     3933   5398     0      2      3   1931     20    372   
264              0     3985   5451     0      2      3   1993     20    372   
...            ...      ...    ...   ...    ...    ...    ...    ...    ...   
3928             0     4001   5467     0      2      3   2012     20    372   
3930             0     3939   5404     0      2      3   1939     20    372   
4001             0     4000   5466     0      2      3   2011     20    372   
4099             0     3946   5411     0      2      3   1946     20    372   
4177             0     4021   5487     0      2      3   2037     20    372   

      pkts  ...   sum   min   max  spkts  dpkts  sb

##Server

In [35]:
import random
class server():
    def __init__(self):
        self.server_model=Net(0.00)
        self.trainset=server_data
        self.testset=testdata_list
        self.clients=[]
        n_clients=8
        config = {
            "server_round": round,
            "local_epochs": 5
        }
        for i in range(n_clients):
            client=client_fn(i,config)
            self.clients.append(client)

    
    #def fit_round():
    def simulation(self,rounds):
        
        for round in range(rounds):
            for i in range(len(self.clients)):
                #kết hợp data sever và xáo trộn
                data = pd.concat([self.clients[i].trainset, self.trainset], axis=0)
                print(f"training client {i} with {len(data)} samples")
                data=data.sample(frac=1).reset_index(drop=True)
                if (i==3 or i==6) and (round==2):
                    semantic_backdoor(data,'dur')
                self.clients[i].trainset= data
                #D_parameters,G_parameters=client.get_parameter()
                results=self.clients[i].fit()
            #train server model

            X = data.iloc[:, 1:].values
            y = data.iloc[:, 0].values

            y=y.squeeze().tolist()
            y=torch.tensor(y)

            self.server_model.fit(X,y,0.003,0.2,15)
            self.server_model.Cross_validation(X,y,5,0.003)

            #evaluate round accuracy
            _,accuracy=self.server_model.evaluate(self.testset[0],self.testset[1])
            print(f"round{round} accuracy:{accuracy}")


            

            


In [36]:
fed=server()

0
thống kê nhãn subcategory :
6 69
7 62
4 74
3 63
5 59
1 77
2 60
0 10
1
thống kê nhãn subcategory :
7 67
6 72
2 59
3 79
1 64
5 60
4 64
0 9
2
thống kê nhãn subcategory :
3 64
2 64
5 74
1 67
4 67
0 7
6 71
7 60
3
thống kê nhãn subcategory :
3 64
7 64
5 75
1 60
2 70
4 69
6 68
0 4
4
thống kê nhãn subcategory :
7 77
5 63
6 63
3 58
1 72
2 77
4 52
0 12
5
thống kê nhãn subcategory :
3 69
1 70
6 63
7 68
2 76
5 74
4 52
0 2
6
thống kê nhãn subcategory :
7 80
5 66
2 66
6 58
0 11
4 80
3 56
1 57
7
thống kê nhãn subcategory :
3 62
1 67
2 62
6 62
7 68
4 75
5 69
0 9


In [37]:
fed.simulation(5)

training client 0 with 479 samples
training client 1 with 479 samples


#Train models with IID and non-IID data  